In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn as sk
import pickle
import copy
from pyts.classification import TimeSeriesForest
from sklearn.model_selection import train_test_split
from pyts.classification import LearningShapelets


In [2]:
AU_COLS = ['AU01_r',
'AU02_r',
'AU04_r',
'AU05_r',
'AU06_r',
'AU07_r',
'AU09_r',
'AU10_r',
'AU12_r',
'AU14_r',
'AU15_r',
'AU17_r',
'AU20_r',
'AU23_r',
'AU25_r',
'AU26_r',
'AU45_r']

ID_COLS = ['filename', "frame", "video_id", "emotion_1_id"]

In [3]:
load_dotenv()

# path to save figures
output_path = os.getenv("VIDEO_OUT")

In [4]:
load_path = os.path.join(output_path, 'video_data_time_series_reduced.csv')
df = pd.read_csv(load_path)

In [5]:
df

,filename,frame,video_id,emotion_1_id,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r
0,A220_reg_p_2,1,A220,0,0.000,0.0,0.420,0.0,0.0,0.137778,...,0.244,0.000000,0.328,0.000,0.119816,0.0,0.168950,0.080,0.120690,0.000000
1,A220_reg_p_2,2,A220,0,0.000,0.0,0.422,0.0,0.0,0.057778,...,0.202,0.000000,0.344,0.030,0.066820,0.0,0.146119,0.082,0.106322,0.020080
2,A220_reg_p_2,3,A220,0,0.000,0.0,0.412,0.0,0.0,0.024444,...,0.176,0.000000,0.348,0.030,0.046083,0.0,0.134703,0.092,0.077586,0.020080
3,A220_reg_p_2,4,A220,0,0.000,0.0,0.412,0.0,0.0,0.057778,...,0.180,0.000000,0.340,0.024,0.062212,0.0,0.139269,0.084,0.063218,0.018072
4,A220_reg_p_2,5,A220,0,0.000,0.0,0.406,0.0,0.0,0.053333,...,0.174,0.000000,0.344,0.020,0.062212,0.0,0.134703,0.096,0.100575,0.010040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98332,A220_anx_v_3,306,A220,34,0.002,0.0,0.124,0.0,0.0,0.104444,...,0.274,0.144550,0.420,0.000,0.179724,0.0,0.216895,0.006,0.000000,0.000000
98333,A220_anx_v_3,307,A220,34,0.006,0.0,0.126,0.0,0.0,0.102222,...,0.284,0.149289,0.416,0.000,0.186636,0.0,0.205479,0.006,0.000000,0.000000
98334,A220_anx_v_3,308,A220,34,0.006,0.0,0.110,0.0,0.0,0.102222,...,0.292,0.151659,0.406,0.000,0.191244,0.0,0.182648,0.004,0.000000,0.000000
98335,A220_anx_v_3,309,A220,34,0.010,0.0,0.100,0.0,0.0,0.075556,...,0.286,0.144550,0.404,0.000,0.184332,0.0,0.162100,0.004,0.000000,0.000000


In [6]:
AU17 = df[["filename", "emotion_1_id", "AU17_r"]]

In [7]:
AU17.head(200)

,filename,emotion_1_id,AU17_r
0,A220_reg_p_2,0,0.119816
1,A220_reg_p_2,0,0.066820
2,A220_reg_p_2,0,0.046083
3,A220_reg_p_2,0,0.062212
4,A220_reg_p_2,0,0.062212
...,...,...,...
195,A220_reg_p_2,0,0.027650
196,A220_reg_p_2,0,0.020737
197,A220_reg_p_2,0,0.006912
198,A220_reg_p_2,0,0.006912


In [8]:
X = []
y = []

In [9]:
for _, group in AU17.groupby('filename'):
    x_list = list(group["AU17_r"])
    X.append(x_list)
    
    y_list = group["emotion_1_id"].unique()
    if len(y_list) == 1:
        y.append(y_list[0])
    else:
        raise ValueError('More than 1 label for time series encountered')

In [10]:
y = np.asarray(y)

In [11]:
length = max(map(len, X))
X=np.array([xi+[0]*(length-len(xi)) for xi in X])

In [12]:
X.shape

(352, 1084)

In [13]:
y.shape

(352,)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

In [20]:
clf = TimeSeriesForest(random_state=43, verbose = 1)
clf.fit(X_train, y_train)

clf.score(X_test, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    7.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.1s finished


0.1111111111111111

### Shapelets

In [21]:
clf = LearningShapelets(random_state=42, tol=0.01, verbose = 51)
clf.fit(X_train, y_train)

clf.score(X_test, y_test)

KeyboardInterrupt: 